In [1]:
VER = 1
import pandas as pd, numpy as np
import pickle, glob, gc

In [2]:
valid = pd.read_parquet('test_with_d.parquet')
valid.head()

,session,aid,ts,type,d
0,11098528,11830,1661119200,0,0
1,11098529,1105029,1661119200,0,0
2,11098530,264500,1661119200,0,0
3,11098530,264500,1661119288,0,0
4,11098530,409236,1661119369,0,0


In [3]:
#MN = 1661119200
MN = valid.ts.min()
print( MN ) #should be 1661119200 for valid
valid['day'] = (valid.ts - MN) // (60*60*24)

In [4]:
valid.head()

,session,aid,ts,type,d,day
0,11098528,11830,1661119200,0,0,0
1,11098529,1105029,1661119200,0,0,0
2,11098530,264500,1661119200,0,0,0
3,11098530,264500,1661119288,0,0,0
4,11098530,409236,1661119369,0,0,0


In [5]:
valid = valid.sort_values(['session','ts'])

In [6]:
valid['x'] = valid.groupby(['session','d']).d.cumcount()

In [7]:
valid['a'] = 0
valid.loc[valid.x==0,'a'] = 1

In [8]:
valid = valid.sort_values(['session','ts'],ascending=[True,False])

In [9]:
valid['x'] = valid.groupby(['session','d']).d.cumcount()

In [10]:
valid['b'] = 0
valid.loc[valid.x==0,'b'] = 1

In [11]:
valid = valid.sort_values(['session','ts'])

In [12]:
valid = valid.drop('x',axis=1)

In [13]:
valid.head()

,session,aid,ts,type,d,day,a,b
0,11098528,11830,1661119200,0,0,0,1,1
1,11098529,1105029,1661119200,0,0,0,1,1
2,11098530,264500,1661119200,0,0,0,1,0
3,11098530,264500,1661119288,0,0,0,0,0
4,11098530,409236,1661119369,0,0,0,0,0


In [14]:
%%time
temp = [group for name, group in valid.sort_values(["session", "ts"]).groupby(["session"])]
print(len(temp))

1801251
CPU times: user 34.8 s, sys: 1.68 s, total: 36.5 s
Wall time: 36.5 s


In [15]:
%%time
PIECES = 10

for PART in range(PIECES):
    print(PART)
    
    print('### PART {} - from {} to {}'.format(PART + 1, PART * len(temp)//PIECES, (PART+1) * len(temp) // PIECES))
    
    mylist = [[h.session.iloc[0], h.aid.to_list(), h.type.to_list(), h.ts.to_list(), h.d.to_list(), 
               h.a.to_list(), h.b.to_list(), h.day.to_list() ] 
              for h in temp[PART * len(temp)//PIECES:(PART+1) * len(temp) // PIECES]]
    
    with open(f'../../data/train_data/lists/valid_group_tolist_{PART}_{VER}.pkl', 'wb') as f:
        pickle.dump(mylist, f) 

0
### PART 1 - from 0 to 180125
1
### PART 2 - from 180125 to 360250
2
### PART 3 - from 360250 to 540375
3
### PART 4 - from 540375 to 720500
4
### PART 5 - from 720500 to 900625
5
### PART 6 - from 900625 to 1080750
6
### PART 7 - from 1080750 to 1260875
7
### PART 8 - from 1260875 to 1441000
8
### PART 9 - from 1441000 to 1621125
9
### PART 10 - from 1621125 to 1801251
CPU times: user 13min 46s, sys: 11.7 s, total: 13min 58s
Wall time: 13min 58s
